# Preparing explainer data
This notebook contains the code to reproduce the figures in the [explainer](TK) describing the methods contained in this repo and the dataset they produced.

In [ ]:
import json

import fsspec
import geopandas as gpd
import numpy as np
import xarray as xr

In [ ]:
path = "s3://carbonplan-climate-impacts/extreme-heat/v1.0/inputs/all_regions_and_cities.json"
with fsspec.open(path) as file:
    regions_df = gpd.read_file(file)

In [ ]:
gcm = "ACCESS-CM2"
city_names = [
    "Bangkok",
    "Karachi",
    "Dubai",
    "Phoenix",
]
city_ids = []
for city in city_names:
    city_ids.append(
        regions_df[regions_df["UC_NM_MN"] == city]["processing_id"].values[0]
    )

Prepare data for Figure 2

In [ ]:
ref = xr.open_zarr(
    "s3://carbonplan-climate-impacts/extreme-heat/v1.0/inputs/wbgt-UHE-daily-historical.zarr/"
)
ref = ref.chunk({"time": -1, "processing_id": 850})

In [ ]:
ts_dict = {}
for scenario in ["historical", "ssp245-2030", "ssp245-2050"]:
    ts_dict[scenario] = xr.open_zarr(
        f"s3://carbonplan-scratch/extreme-heat/wbgt-shade-regions/{gcm}-{scenario}-bc.zarr"
    ).WBGT.chunk({"time": -1})
    ts_dict[f"{scenario}-bc"] = xr.open_zarr(
        f"s3://carbonplan-scratch/extreme-heat/wbgt-shade-regions/{gcm}-{scenario}-bc.zarr"
    ).scen.chunk({"time": -1})

In [ ]:
time_slice_dict = {
    "historical": slice("1985", "2014"),
    "ssp245-2030": slice("2020", "2039"),
    "ssp245-2050": slice("2040", "2059"),
}

In [ ]:
data_to_plot = {}
for city_id, city_name in zip(city_ids, city_names):
    city_name = regions_df[regions_df["processing_id"] == city_id]["UC_NM_MN"].values[0]
    data_to_plot[city_name] = {}
    data_to_plot[city_name]["ref"] = list(
        ref.WBGT.sel(processing_id=city_id)
        .sel(time=time_slice_dict["historical"])
        .groupby("time.dayofyear")
        .mean()
        .load()
        .values
    )
    for scenario, time_slice in time_slice_dict.items():
        data_to_plot[city_name][scenario] = list(
            ts_dict[scenario]
            .sel(processing_id=city_id)
            .sel(time=time_slice)
            .groupby("time.dayofyear")
            .mean()
            .load()
            .values
        )
        data_to_plot[city_name][scenario + "-bc"] = list(
            ts_dict[scenario + "-bc"]
            .sel(processing_id=city_id)
            .sel(time=time_slice)
            .groupby("time.dayofyear")
            .mean()
            .load()
            .values
        )

In [ ]:
with fsspec.open(
    "s3://carbonplan-climate-impacts/extreme-heat/v1.0/outputs/web/explainer/bias_correction.json",
    "w",
) as fp:
    json.dump(data_to_plot, fp)

Prepare data for Figure 3

In [ ]:
gcm = "ACCESS-CM2"
ts_dict = {}
for scenario in ["historical", "ssp245-2030", "ssp245-2050"]:
    ts_dict[f"{scenario}-shade"] = xr.open_zarr(
        tk change to f"s3://carbonplan-scratch/extreme-heat/wbgt-shade-regions/{gcm}-{scenario}-bc.zarr"
    ).scen.chunk({"time": -1})
    ts_dict[f"{scenario}-sun"] = xr.open_zarr(
        tk change to f"s3://carbonplan-scratch/extreme-heat/wbgt-sun-regions/{gcm}-{scenario}.zarr"
    ).outdoor_WBGT.chunk({"time": -1})

In [ ]:
data_to_plot = {}
for city_id, city_name in zip(city_ids, city_names):
    data_to_plot[city_name] = {}
    for variable in ["shade", "sun"]:
        data_to_plot[city_name][f"historical-{variable}"] = {}
        for year in np.arange(1985, 2015):
            data_to_plot[city_name][f"historical-{variable}"][str(year)] = list(
                ts_dict[f"historical-{variable}"]
                .sel(processing_id=city_id, time=slice(str(year), str(year)))
                .values
            )
        data_to_plot[city_name][f"ssp245-2030-{variable}"] = {}
        for year in np.arange(2020, 2040):
            data_to_plot[city_name][f"ssp245-2030-{variable}"][str(year)] = list(
                ts_dict[f"ssp245-2030-{variable}"]
                .sel(processing_id=city_id, time=slice(str(year), str(year)))
                .values
            )
        data_to_plot[city_name][f"ssp245-2050-{variable}"] = {}
        for year in np.arange(2040, 2060):
            data_to_plot[city_name][f"ssp245-2050-{variable}"][str(year)] = list(
                ts_dict[f"ssp245-2050-{variable}"]
                .sel(processing_id=city_id, time=slice(str(year), str(year)))
                .values
            )

Prepare data for Figure 4

In [ ]:
cities = regions_df[
    (~regions_df["ID_HDC_G0"].isnull() | ~regions_df["UACE20"].isnull())
]

In [ ]:
scenarios = ["historical", "ssp245-2030", "ssp245-2050"]
map_json = {}
for variable in ["WBGT-sun", "WBGT-shade"]:
    map_json[variable] = {}
    for scenario in scenarios:
        map_json[variable][scenario] = {}
        ds = xr.open_zarr(
            f"s3://carbonplan-climate-impacts/extreme-heat/v1.0/outputs/zarr/summaries/{scenario}-summaries-{variable}.zarr"
        )
        for threshold in [29, 30.5, 32]:
            df = (
                ds.sel(
                    processing_id=cities["processing_id"].values, gcm="multimodel_media"
                )[f"days_exceeding_{threshold}degC"]
                .to_dataframe()
                .drop("gcm", axis=1)
            )
            data = np.squeeze(df.values)
            map_json[variable][scenario][str(threshold)] = list(data)

In [ ]:
lon = cities.centroid.x.values
lat = cities.centroid.y.values

In [ ]:
map_json["lat"] = list(lat)
map_json["lon"] = list(lon)

In [ ]:
with fsspec.open(
    "s3://carbonplan-climate-impacts/extreme-heat/v1.0/outputs/web/explainer/maps.json",
    "w",
) as fp:
    json.dump(map_json, fp)